In [3]:
import warnings
import os
from dotenv import load_dotenv

warnings.filterwarnings('ignore')
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'

# Load environment variables from .env file
load_dotenv()

True

Imports essential modules from CrewAI.

In [4]:
from crewai import Agent, Task, Crew, Process
from crewai.llm import LLM
from crewai_tools import TavilySearchTool

# Access the variables
gemini_api_key = os.getenv("GEMINI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

os.environ["GEMINI_API_KEY"] = gemini_api_key
os.environ["TAVILY_API_KEY"] = tavily_api_key

gemini_llm = LLM(
    model="gemini/gemini-2.0-flash",
    temperature=0,
)

# Run Local LLM to escape API quota limitations
llm = LLM(model="ollama/tinyllama")

# Initialize TavilySearchTool
search_tool = TavilySearchTool()

Sets up the conditional task checker

In [5]:
from typing import List
from pydantic import BaseModel
from crewai.tasks.conditional_task import ConditionalTask
from crewai.tasks.task_output import TaskOutput

# Define a function to assess whether the data needs to be augmented
def should_fetch_more_data(output: TaskOutput) -> bool:
    return len(output.pydantic.events) < 3  # Condition to trigger task

Agents declared

In [6]:
# Create agents for different roles
data_collector = Agent(
    role="Data Collector",
    goal="Retrieve event data using Tavily Search Tool",
    backstory="You have a knack for finding the most exciting events happening around.",
    verbose=True,
    tools=[search_tool],
    llm=gemini_llm,
    max_iter=3,
)

data_analyzer = Agent(
    role="Data Analyzer",
    goal="Analyze the collected data",
    backstory="You're known for your analytical skills, making sense of complex datasets.",
    verbose=True,
    llm=gemini_llm,
    max_iter=3,
)

summary_creator = Agent(
    role="Summary Creator",
    goal="Produce a concise summary from the event data",
    backstory="You're a skilled writer, able to summarize information clearly and effectively.",
    verbose=True,
    llm=gemini_llm,
    max_iter=3,
)

Tasks are declared as we did before however the conditional task for this example is declared with ConditionalTask() instead

In [7]:
class EventsData(BaseModel):
    events: List[str]

# Define the tasks
fetch_task = Task(
    description="Collect event data for Bangalore, India using Serper tool",
    expected_output="A list of 3 exciting events happening in Bangalore, India this week",
    agent=data_collector,
    output_pydantic=EventsData,
)

verify_data_task = ConditionalTask(
    description="""
        Ensure that sufficient event data has been collected.
        If fewer than 3 events are found, gather more using the Serper tool.
        """,
    expected_output="An updated list of at least 3 events happening in Bangalore, India this week",
    condition=should_fetch_more_data,
    agent=data_analyzer,
)

summary_task = Task(
    description="Summarize the collected events data for Bangalore, India",
    expected_output="summary_generated",
    agent=summary_creator,
)

In [9]:
# Assemble the crew with the defined agents and tasks
crew = Crew(
    agents=[data_collector, data_analyzer, summary_creator],
    tasks=[fetch_task, verify_data_task, summary_task],
    verbose=True,
    planning=True,  # Retain the planning feature
    planning_llm=gemini_llm,
)

# Execute the tasks with the crew
result = crew.kickoff()
print("results", result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: e62edc2a-76f7-4c7a-8113-555e347e86bc                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

 
[2025-08-22 17:47:04][INFO]: Planning the crew execution


╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 59e5257b-2d2c-4e34-bafe-55b52fb95f0c                                                                     │
│  Agent: Task Execution Planner                                                                                  │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Collector                                                                                          │
│                                                                                                                 │
│  Task: Collect event data for Bangalore, India using Serper tool1. The Data Collector agent will use the        │
│  Tavily Search tool to search for events happening in Bangalore, India this week. The search query will be      │
│  'events in Bangalore India this week'.                                                                         │
│  2. The Data Collector agent will set the Tavily Search tool's parameters to maximize the chances of finding    │
│  relevant events. Specifically, it will set the 'days' parameter to 7 to focus on events within the current     │
│  week.                                                                                                          │
│  3. The Data Collector agent will execute the Tavily Search tool and retrieve the search results.               │
│  4. The Data Collector agent will parse the search results to identify potential events, extracting relevant    │
│  information such as event name, date, time, location, and a brief description.                                 │
│  5. The Data Collector agent will compile a list of the identified events.                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Data Collector                                                                                          │
│                                                                                                                 │
│  Thought: I need to find exciting events happening in Bangalore, India this week. I will use the Tavily Search  │
│  tool to search for events and then compile a list of the most exciting ones.                                   │
│                                                                                                                 │
│  Using Tool: Tavily Search                                                                                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summary Creator                                                                                         │
│                                                                                                                 │
│  Task: Summarize the collected events data for Bangalore, India1. The Summary Creator agent will receive the    │
│  final list of events from the Data Analyzer agent.                                                             │
│  2. The Summary Creator agent will iterate through the list of events.                                          │
│  3. For each event, the Summary Creator agent will extract the event name, date, time, location, and a brief    │
│  description.                                                                                                   │
│  4. The Summary Creator agent will generate a concise summary of each event, highlighting the key details.      │
│  5. The Summary Creator agent will combine the individual event summaries into a single, coherent summary of    │
│  events happening in Bangalore, India this week.                                                                │
│  6. The Summary Creator agent will ensure the summary is clear, concise, and easy to understand.                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Summary Creator                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  This week in Bangalore, India, catch "Late Night Sexy Jokes - Best Of Bangalore" (BookMyShow) for an evening   │
│  of comedy. Tej Music Vodka will be live at Sunburn Union (BookMyShow), offering a musical performance. For     │
│  entrepreneurs and startup enthusiasts, eChai's Startup Open House in Bengaluru (Eventbrite) provides a         │
│  networking and learning opportunity.                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: fd341580-0048-4937-9a5e-748f31c433ab                                                                     │
│  Agent: Summary Creator                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: e62edc2a-76f7-4c7a-8113-555e347e86bc                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: This week in Bangalore, India, catch "Late Night Sexy Jokes - Best Of Bangalore" (BookMyShow)    │
│  for an evening of comedy. Tej Music Vodka will be live at Sunburn Union (BookMyShow), offering a musical       │
│  performance. For entrepreneurs and startup enthusiasts, eChai's Startup Open House in Bengaluru (Eventbrite)   │
│  provides a networking and learning opportunity.                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

results This week in Bangalore, India, catch "Late Night Sexy Jokes - Best Of Bangalore" (BookMyShow) for an evening of comedy. Tej Music Vodka will be live at Sunburn Union (BookMyShow), offering a musical performance. For entrepreneurs and startup enthusiasts, eChai's Startup Open House in Bengaluru (Eventbrite) provides a networking and learning opportunity.


In [10]:
from IPython.display import Markdown

# Convert the CrewOutput object to a Markdown string
result_markdown = result.raw

# Display the result as Markdown
Markdown(result_markdown)

This week in Bangalore, India, catch "Late Night Sexy Jokes - Best Of Bangalore" (BookMyShow) for an evening of comedy. Tej Music Vodka will be live at Sunburn Union (BookMyShow), offering a musical performance. For entrepreneurs and startup enthusiasts, eChai's Startup Open House in Bengaluru (Eventbrite) provides a networking and learning opportunity.